In [1]:
##Function for splitting the field goal variables..
def fg_split(a):
  a = a.replace("(","-").replace(" ","").replace(")","").replace("%","").split("-")
  return a

def fg_cleaning(data_frame):
    
    v = data_frame["FIELD GOALS 2PM-A"].apply(fg_split)
    FG2PM = [float(v[i][0]) for i in range(len(v))]
    FG2PA = [float(v[i][1]) for i in range(len(v))]

    v1 = data_frame["FIELD GOALS 3PM-A"].apply(fg_split)
    FG3PM = [float(v1[i][0]) for i in range(len(v1))]
    FG3PA = [float(v1[i][1]) for i in range(len(v1))]

    v2 = data_frame["FIELD GOALS FTM-A"].apply(fg_split)
    FGFTMM = [float(v2[i][0]) for i in range(len(v2))]
    FGFTMA = [float(v2[i][1]) for i in range(len(v2))]



    data_frame["FG2PM"] = FG2PM
    data_frame["FG2PA"] = FG2PA

    data_frame["FG3PM"] = FG3PM
    data_frame["FG3PA"] = FG3PA

    data_frame["FTM"] = FGFTMM
    data_frame["FTA"] = FGFTMA

    data_frame["FGM"] = data_frame["FG2PM"]+data_frame["FG3PM"]
    data_frame["FGA"] = data_frame["FG2PA"]+data_frame["FG3PA"]

    data_frame.drop(columns = ["FIELD GOALS 2PM-A","FIELD GOALS 3PM-A","FIELD GOALS FTM-A","RNK","+/-"],axis = 1, inplace = True)
    data_frame.columns = [c.replace(" ","_") for c in data_frame.columns]
    data_frame["MATCH"] = data_frame["MATCH"].astype("str")
    data_frame["ID"] = data_frame["TEAM1"]+"_"+data_frame["MATCH"]


    #Converting the datatypes of varibales from object to float
    data_frame.MIN = data_frame.MIN.astype("float")
    data_frame.REBOUNDS_OFF = data_frame.REBOUNDS_OFF.astype("float")
    data_frame.REBOUNDS_DEF = data_frame.REBOUNDS_DEF.astype("float")
    data_frame.REBOUNDS_RB = data_frame.REBOUNDS_RB.astype("float")
    data_frame.AS = data_frame.AS.astype("float")
    data_frame.FOULS_F = data_frame.FOULS_F.astype("float")
    data_frame.FOULS_RV = data_frame.FOULS_RV.astype("float")
    data_frame.ST = data_frame.ST.astype("float")
    data_frame.BLOCKS_FV = data_frame.BLOCKS_FV.astype("float")
    data_frame.BLOCKS_AG = data_frame.BLOCKS_AG.astype("float")
    data_frame.TO = data_frame.TO.astype("float")
    data_frame.PT = data_frame.PT.astype("float")
    
    return data_frame

In [1]:
##Chinese Basketball - CBA_2021-22_boxscore - Match_list
##Polish Basketball league - CBA_2021-22_boxscore - Match_list
##Euro League - EuroLeague - Euro_2021-22_boxscore - Match_list
##Importing the libraries needed

import pandas as pd
import numpy as np
import httplib2
import requests
import openpyxl
from bs4 import BeautifulSoup, SoupStrainer


In [46]:
col_names = ['JERSEY NO', 'NAME', 'MIN', 'FIELD GOALS 2PM-A', 'FIELD GOALS 3PM-A',
       'FIELD GOALS FTM-A', 'REBOUNDS OFF', 'REBOUNDS DEF', 'REBOUNDS RB',
       'AS', 'FOULS F', 'FOULS RV', 'ST', 'BLOCKS FV', 'BLOCKS AG', 'TO', 'PT',
       'RNK', '+/-']


input1  = int(input("Do you want to scrape data for :\n 1. For a new League \n 2. Append to a league data\n\n"))
if input1 == 1:
    url1 = input("Paste the URL of the league you want to scrape..!!\n\n")
    file = input("Enter the file name you want to give...")
    m = []
    http = httplib2.Http()
    status, response = http.request(url1)
    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
          m.append(link['href'])
          m = [m for m in m if "Box-Score" in m]
            
    print("Links have been collected....Scraping is being done..")
    master_df = pd.DataFrame(columns = col_names)
    j = 1
    for i in m:
      df = pd.read_html(requests.get(i).content)
      t1 = pd.DataFrame(df[0])
      t1_BS = pd.DataFrame(df[1])
      t2 = pd.DataFrame(df[2])
      t2_BS = pd.DataFrame(df[3])
      t1_BS.columns = col_names
      t2_BS.columns = col_names
      u = str(t1).replace(" ","")
      u2 = str(t2).replace(" ","")
      t1_BS["TEAM1"] = u[3:u.find("(")]
      t2_BS["TEAM1"] = u2[3:u2.find("(")]
      df = t1_BS.append(t2_BS, ignore_index = True)
      df["MATCH"] = j
      j = j+1
      master_df = master_df.append(df, ignore_index = True)
    
    print("Scraping of the data has been done...!!")
    print("Data is being cleaned...")

    ##Cleaning the data
    master_df = master_df[-master_df["NAME"].isin(["Total",""])]
    master_df = master_df[-master_df["MIN"].isin(["DNP - Coach's Decision",""])]
    master_df = master_df[-master_df["MIN"].isnull()]
    master_df = master_df.reset_index(inplace = False,drop = True)

    print("Data has been cleaned....")
    
    data_frame = fg_cleaning(master_df)
    
    with pd.ExcelWriter(file,engine ='openpyxl') as writer:
        data_frame.to_excel(writer,sheet_name = "BoxScores")
        #player_stats.to_excel(writer,sheet_name='PlayerStats')
        pd.DataFrame(m,columns = ["Match_list"]).to_excel(writer,sheet_name='Matchlist')
    
    
if input1 == 2:
    
    file = input("Enter the file name...")
    loaded_file = openpyxl.load_workbook(file)
    names_sh = loaded_file.get_sheet_names()
    prior_data1 = loaded_file[names_sh[0]]
    prior_data = pd.DataFrame(prior_data1.values)
    prior_matches1 = loaded_file[names_sh[2]]
    prior_matches = pd.DataFrame(prior_matches1.values)
    
    url1 = input("Paste the URL of the league you want to scrape..!!\n\n")
    m1 = []
    http = httplib2.Http()
    status, response = http.request(url1)
    for link in BeautifulSoup(response, 'html.parser',parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
          m1.append(link['href'])
          m1 = [m1 for m1 in m1 if "Box-Score" in m1]
        
    m = list(set(m1) - set(list(prior_matches.iloc[:,1])))        

    master_df = pd.DataFrame(columns = col_names)
    j = len(prior_matches) -1
    for i in m:
      df = pd.read_html(requests.get(i).content)
      t1 = pd.DataFrame(df[0])
      t1_BS = pd.DataFrame(df[1])
      t2 = pd.DataFrame(df[2])
      t2_BS = pd.DataFrame(df[3])
      t1_BS.columns = col_names
      t2_BS.columns = col_names
      u = str(t1).replace(" ","")
      u2 = str(t2).replace(" ","")
      t1_BS["TEAM1"] = u[3:u.find("(")]
      t2_BS["TEAM1"] = u2[3:u2.find("(")]
      df = t1_BS.append(t2_BS, ignore_index = True)
      df["MATCH"] = j
      j = j+1
      master_df = master_df.append(df, ignore_index = True)
    
    master_df = master_df[-master_df["NAME"].isin(["Total",""])]
    master_df = master_df[-master_df["MIN"].isin(["DNP - Coach's Decision",""])]
    master_df = master_df[-master_df["MIN"].isnull()]
    master_df = master_df.reset_index(inplace = False,drop = True)
    
    master_df1 = fg_cleaning(master_df)
    

    prior_match = pd.read_excel(file,sheet_name = "BoxScores")
    data_frame = prior_match.append(master_df1)
 
    with pd.ExcelWriter(file,engine ='openpyxl') as writer:
        data_frame.to_excel(writer,sheet_name = "BoxScores")
        #player_stats.to_excel(writer,sheet_name='PlayerStats')
        pd.DataFrame(m1,columns = ["Match_list"]).to_excel(writer,sheet_name='Matchlist')
        


    



    
    

Do you want to scrape data for :
 1. For a new League 
 2. Append to a league data

1
Paste the URL of the league you want to scrape..!!

https://www.eurobasket.com/ABA-League/games-schedule.aspx?SectionID=148&League=1&Season=2021-2022&LName=ABA-League
Enter the file name you want to give...ABA_LEAGUE_2021-22_boxscore.xlsx
Links have been collected....Scraping is being done..
Scraping of the data has been done...!!
Data is being cleaned...
Data has been cleaned....


In [2]:
data_frame = pd.DataFrame("NBA_2021-22_boxscorev2.xlsx",sheet_name = "Player_Stats")

TypeError: __init__() got an unexpected keyword argument 'sheet_name'

In [47]:
team_data = data_frame.groupby(["ID"])[['NAME', 'MIN', 'REBOUNDS_OFF', 'REBOUNDS_DEF', 'REBOUNDS_RB', 'AS','FOULS_F', 'FOULS_RV', 'ST', 'BLOCKS_FV', 'BLOCKS_AG', 'TO', 'PT','TEAM1', 'MATCH', 'FG2PM', 'FG2PA', 'FG3PM', 'FG3PA', 'FTM', 'FTA','FGM', 'FGA']].sum().round(2)

team_data.reset_index(inplace = True)

In [48]:
v = list(team_data.columns)
v.remove("ID")
v = ["tm"+i for i in v]
v.insert(0,"ID")
team_data.columns = v
team_data.head()

,ID,tmMIN,tmREBOUNDS_OFF,tmREBOUNDS_DEF,tmREBOUNDS_RB,tmAS,tmFOULS_F,tmFOULS_RV,tmST,tmBLOCKS_FV,...,tmTO,tmPT,tmFG2PM,tmFG2PA,tmFG3PM,tmFG3PA,tmFTM,tmFTA,tmFGM,tmFGA
0,BORACCA_12.0,203.0,12.0,20.0,32.0,14.0,20.0,22.0,10.0,0.0,...,12.0,76.0,14.0,44.0,11.0,27.0,15.0,22.0,25.0,71.0
1,BORACCA_18.0,203.0,16.0,25.0,41.0,18.0,18.0,26.0,4.0,3.0,...,17.0,90.0,17.0,37.0,12.0,25.0,20.0,24.0,29.0,62.0
2,BORACCA_25.0,205.0,8.0,29.0,37.0,14.0,19.0,20.0,9.0,2.0,...,14.0,80.0,16.0,37.0,14.0,30.0,6.0,14.0,30.0,67.0
3,BORACCA_3.0,207.0,10.0,21.0,31.0,25.0,22.0,14.0,3.0,0.0,...,13.0,89.0,20.0,40.0,13.0,31.0,10.0,13.0,33.0,71.0
4,BORACCA_33.0,204.0,7.0,23.0,30.0,7.0,21.0,17.0,10.0,1.0,...,17.0,56.0,11.0,31.0,8.0,27.0,10.0,13.0,19.0,58.0


In [40]:
data_frame.head()

,JERSEY_NO,NAME,MIN,REBOUNDS_OFF,REBOUNDS_DEF,REBOUNDS_RB,AS,FOULS_F,FOULS_RV,ST,...,MATCH,FG2PM,FG2PA,FG3PM,FG3PA,FTM,FTA,FGM,FGA,ID
0,5,"Micov, Vladimir",31.0,1.0,5.0,6.0,0.0,2.0,2.0,2.0,...,30.0,3.0,6.0,1.0,5.0,2.0,4.0,4.0,11.0,BUDUCNOST_30.0
1,1,"Cobbs, Justin",30.0,1.0,2.0,3.0,4.0,3.0,6.0,2.0,...,30.0,9.0,14.0,0.0,2.0,5.0,6.0,9.0,16.0,BUDUCNOST_30.0
2,19,"Nikolic, Zoran",27.0,4.0,2.0,6.0,1.0,3.0,5.0,1.0,...,30.0,3.0,5.0,0.0,0.0,3.0,6.0,3.0,5.0,BUDUCNOST_30.0
3,24,"Wiley, Jacob",17.0,4.0,2.0,6.0,1.0,1.0,0.0,0.0,...,30.0,2.0,5.0,0.0,0.0,0.0,0.0,2.0,5.0,BUDUCNOST_30.0
4,3,"Atic, Edin",16.0,1.0,2.0,3.0,0.0,0.0,1.0,0.0,...,30.0,1.0,3.0,0.0,1.0,1.0,2.0,1.0,4.0,BUDUCNOST_30.0


In [49]:
fs = pd.merge(data_frame,team_data, how = "left", on = "ID")
fs["qAST"] = round(((fs.MIN/(fs.tmMIN/5)) * (1.14 * ((fs.tmAS - fs.AS)/fs.tmFGM))) + ((((fs.tmAS/fs.tmMIN)*fs.MIN*5-fs.AS)/((fs.tmFGM/fs.tmMIN)*fs.MIN*5-fs.tmFGM))*(1-(fs.MIN/(fs.tmMIN/5)))),2)
fs["FG_Part"] = round(fs.FGM * (1-0.5*((fs.PT - fs.FTM)/(2*fs.FGA))*fs.qAST),2)
fs["AST_Part"] = round(0.5*(((fs.tmPT - fs.tmFTM) - (fs.PT - fs.FTM))/(2*(fs.tmFGA - fs.FGA)))*fs.AS,2)
fs["FT_Part"] = round((1-(1-(fs.FTM/fs.FTA))**2)*0.4*fs.FTA,2)
fs["Team_Scoring_pass"] = round(fs.tmFGM+(1-(1-(fs.tmFTM/fs.tmFTA))**2)*fs.tmFTA*0.4,2)
fs["Team_Play_per"] = round(fs.Team_Scoring_pass/(fs.tmFGA+fs.tmFTA*0.4+fs.tmTO),2)

fs.head()


,JERSEY_NO,NAME,MIN,REBOUNDS_OFF,REBOUNDS_DEF,REBOUNDS_RB,AS,FOULS_F,FOULS_RV,ST,...,tmFTM,tmFTA,tmFGM,tmFGA,qAST,FG_Part,AST_Part,FT_Part,Team_Scoring_pass,Team_Play_per
0,20,"Ivanovic, Nikola",26.0,3.0,3.0,6.0,6.0,2.0,4.0,4.0,...,7.0,9.0,40.0,63.0,0.11,7.64,2.08,0.8,43.42,0.54
1,14,"Hollins, Austin",23.0,0.0,2.0,2.0,4.0,1.0,0.0,2.0,...,7.0,9.0,40.0,63.0,0.09,3.89,1.46,NaN,43.42,0.54
2,12,"Kalinic, Nikola",23.0,0.0,3.0,3.0,6.0,2.0,3.0,0.0,...,7.0,9.0,40.0,63.0,0.11,3.84,2.13,NaN,43.42,0.54
3,33,"Zirbes, Maik",22.0,1.0,5.0,6.0,3.0,2.0,3.0,0.0,...,7.0,9.0,40.0,63.0,0.08,5.79,1.04,NaN,43.42,0.54
4,9,"Mitrovic, Luka",19.0,0.0,2.0,2.0,3.0,2.0,1.0,2.0,...,7.0,9.0,40.0,63.0,0.08,2.94,1.11,NaN,43.42,0.54


In [50]:
dff = data_frame.groupby(["MATCH","TEAM1","ID"])[['REBOUNDS_OFF', 'REBOUNDS_DEF', 'REBOUNDS_RB',"FGM","FGA","TO","PT","FTA","FTM","MIN"]].sum().ffill()
dff.reset_index(inplace = True)
dff.fillna(0,inplace = True)

dummy = pd.DataFrame(columns = dff.columns)
dffff = pd.DataFrame(columns = dff.columns)
for i in dff.MATCH.unique():
    dff1 = dff[dff["MATCH"] == i]
    x = dff1.iloc[0,3:13].values
    y = dff1.iloc[1,3:13].values
    dff1.iloc[0,3:13] = np.array(y)
    dff1.iloc[1,3:13] = np.array(x)
    dfff = dummy.append(dff1,ignore_index = True)
    dffff = dffff.append(dfff,ignore_index = True)
    
    
dffff.rename(columns = {"REBOUNDS_OFF" : "OREBOUNDS_OFF","REBOUNDS_DEF" : "OREBOUNDS_DEF",
                        "FGM":"OFGM","FGA":"OFGA","TO":"OTO","PT":"OPT","FTA":"OFTA",
                        "FTM":"OFTM","MIN":"OMIN"}, inplace = True)

C:\Users\SHIVA\anaconda3\lib\site-packages\pandas\core\indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [51]:
dffff.drop(columns = ["MATCH","TEAM1"],inplace = True)
fs_final = pd.merge(fs,dffff,on = "ID",how = "left")
fs_final.fillna(0,inplace = True)
fs_final["Team_ORB_per"] = fs_final.tmREBOUNDS_OFF/(fs_final.tmREBOUNDS_OFF+fs_final.OREBOUNDS_DEF)
fs_final.fillna(0,inplace = True)
fs_final["Team_ORB_Wgt"] = ((1-fs_final.Team_ORB_per)*fs_final.Team_Play_per)/((1-fs_final.Team_Play_per)*fs_final.Team_ORB_per)
fs_final.fillna(0,inplace = True)
fs_final["ORB_Part"] = fs_final.REBOUNDS_OFF*fs_final.Team_ORB_Wgt*fs_final.Team_Play_per
fs_final["Scposs"] = (fs_final.FG_Part+fs_final.AST_Part+fs_final.FT_Part)*(1-(fs_final.tmREBOUNDS_OFF/fs_final.Team_Scoring_pass)*fs_final.Team_ORB_Wgt*fs_final.Team_Play_per)+fs_final.ORB_Part
fs_final.fillna(0,inplace = True)
fs_final["FGxPoss"] = (fs_final.FGA - fs_final.FGM)*(1-1.07*fs_final.Team_ORB_per)
fs_final["FTxPoss"] = ((1-(fs_final.FTM/fs_final.FTA))**2)*0.4*fs_final.FTA
fs_final.fillna(0,inplace = True)
fs_final["TotPoss"] = fs_final.Scposs+fs_final.FGxPoss+fs_final.FTxPoss+fs_final.TO

##Points Produced calculations
fs_final["PProd_FG_Part"] = 2*(fs_final.FGM+0.5*fs_final.FG3PM)*(1-0.5*((fs_final.PT - fs_final.FTM)/(2*fs_final.FGA))*fs_final.qAST)
fs_final.fillna(0,inplace = True)
fs_final["PProd_AST_Part"] = 2*((fs_final.tmFGM - fs_final.FGM + 0.5*(fs_final.tmFG3PM - fs_final.FG3PM))/(fs_final.tmFGM-fs_final.FGM))*0.5*(((fs_final.tmPT - fs_final.tmFTM)-(fs_final.PT - fs_final.FTM))/(2*(fs_final.tmFGA - fs_final.FGA)))*fs_final.AS
fs_final.fillna(0,inplace = True)
fs_final["PProd_ORB_Part"] = fs_final.REBOUNDS_OFF * fs_final.Team_ORB_Wgt * fs_final.Team_Play_per*(fs_final.tmPT/(fs_final.tmFGM + (1-(1-(fs_final.tmFTM/fs_final.tmFTA))**2)*0.4*fs_final.tmFTA))
fs_final.fillna(0,inplace = True)
fs_final["PProd"] = (fs_final.PProd_FG_Part+fs_final.PProd_AST_Part+fs_final.FTM)*(1-(fs_final.tmREBOUNDS_OFF/fs_final.Team_Scoring_pass)*fs_final.Team_ORB_Wgt*fs_final.Team_Play_per)+fs_final.PProd_ORB_Part
fs_final.fillna(0,inplace = True)
fs_final["Off_Rating"] = (fs_final.PProd/fs_final.TotPoss)*100
fs_final.fillna(0,inplace = True)
fs_final["Floor_per"] = (fs_final.Scposs/fs_final.TotPoss)*100
fs_final.fillna(0,inplace = True)

def_rating = pd.DataFrame()

def_rating["DOR_per"] = fs_final.OREBOUNDS_DEF/(fs_final.OREBOUNDS_OFF+fs_final.tmREBOUNDS_DEF)
def_rating.fillna(0, inplace = True)
def_rating["NAME"] = fs_final.NAME
def_rating["TEAM1"] = fs_final.TEAM1
def_rating["DFG_per"] = fs_final.OFGM/fs_final.OFGA
def_rating.fillna(0,inplace = True)
def_rating["FMwt"] = (def_rating.DFG_per*(1-def_rating.DOR_per))/(def_rating.DFG_per*(1-def_rating.DOR_per)+def_rating.DOR_per*(1-def_rating.DFG_per))
def_rating.fillna(0,inplace = True)
def_rating["Stops1"] = fs_final.ST+fs_final.BLOCKS_FV*def_rating.FMwt*(1-1.07*def_rating.DOR_per)+fs_final.REBOUNDS_DEF*(1-def_rating.FMwt)
def_rating.fillna(0,inplace = True)
def_rating["Stops2"] = (((fs_final.OFGA - fs_final.OFGM - fs_final.tmBLOCKS_FV)/fs_final.tmMIN)*def_rating.FMwt*(1-1.07*def_rating.DOR_per)+((fs_final.OTO-fs_final.tmST)/fs_final.tmMIN))*fs_final.MIN+(fs_final.FOULS_F/fs_final.tmFOULS_F)*0.4*fs_final.OFTA*(1-(fs_final.OFTM/fs_final.OFTA))**2
def_rating.fillna(0,inplace = True)
def_rating["Team_possessions"] = fs_final.tmFGA-fs_final.tmREBOUNDS_OFF+fs_final.tmTO+0.4*fs_final.tmFTA
def_rating.fillna(0,inplace = True)
def_rating["Team_Def_Rating"] = 100*(fs_final.OPT/def_rating.Team_possessions)
def_rating.fillna(0,inplace = True)
def_rating["Stop_per"] = ((def_rating.Stops1+def_rating.Stops2)*fs_final.OMIN)/(def_rating.Team_possessions*fs_final.MIN)
def_rating.fillna(0,inplace = True)
def_rating["D_Pts_per_Scposs"] = fs_final.OPT/(fs_final.OFGM+(1-(1-(fs_final.OFTM/fs_final.OFTA))**2)*fs_final.OFTA*0.4)
def_rating.fillna(0,inplace = True)
def_rating["DRtg"] = def_rating.Team_Def_Rating+0.2*(100*def_rating.D_Pts_per_Scposs*(1-def_rating.Stop_per)-def_rating.Team_Def_Rating)
def_rating.fillna(0,inplace = True)

def_rating["ID"] = fs_final["ID"]
def_rating.head()

,DOR_per,NAME,TEAM1,DFG_per,FMwt,Stops1,Stops2,Team_possessions,Team_Def_Rating,Stop_per,D_Pts_per_Scposs,DRtg,ID
0,0.441176,"Ivanovic, Nikola",CRVENAZVEZDA,0.381818,0.438944,5.683168,1.915183,73.6,82.880435,0.810025,2.234432,74.794076,CRVENAZVEZDA_1.0
1,0.441176,"Hollins, Austin",CRVENAZVEZDA,0.381818,0.438944,3.122112,1.690538,73.6,82.880435,0.579974,2.234432,85.074726,CRVENAZVEZDA_1.0
2,0.441176,"Kalinic, Nikola",CRVENAZVEZDA,0.381818,0.438944,1.683168,1.695299,73.6,82.880435,0.407140,2.234432,92.798436,CRVENAZVEZDA_1.0
3,0.441176,"Zirbes, Maik",CRVENAZVEZDA,0.381818,0.438944,3.268754,1.622005,73.6,82.880435,0.616178,2.234432,83.456852,CRVENAZVEZDA_1.0
4,0.441176,"Mitrovic, Luka",CRVENAZVEZDA,0.381818,0.438944,3.353849,1.402121,73.6,82.880435,0.693806,2.234432,79.987758,CRVENAZVEZDA_1.0


In [52]:
fs_final["Approx_Value"] = fs_final.PT+fs_final.REBOUNDS_RB_x+fs_final.AS+fs_final.ST+fs_final.BLOCKS_FV-fs_final.TO-fs_final.FG3PA-fs_final.FG2PA+fs_final.FG2PM+fs_final.FG3PM-fs_final.FTM+fs_final.FTA
fs_final["Approx_Value1"] = fs_final.PT+1.2*fs_final.REBOUNDS_RB_x+1.5*fs_final.AS+3*fs_final.ST+3*fs_final.BLOCKS_FV-fs_final.TO
fs_final["Approx_Value2"] = 2*fs_final.PT+3*fs_final.REBOUNDS_RB_x+4*fs_final.AS+5*fs_final.ST+5*fs_final.BLOCKS_FV-2*fs_final.TO
fs_final["Approx_Value3"] = 2*fs_final.PT+2.5*fs_final.REBOUNDS_RB_x+3*fs_final.AS+6*fs_final.ST+6*fs_final.BLOCKS_FV-2*fs_final.TO



fs_final["App_value"] = round((fs_final["Approx_Value"]**0.75)/21,2)
fs_final["D11_App_value"] = round((fs_final["Approx_Value1"]**0.75)/21,2)
fs_final["PP_App_value"] = round((fs_final["Approx_Value2"]**0.75)/21,2)
fs_final["FF_App_value"] = round((fs_final["Approx_Value3"]**0.75)/21,2)
rating1 = def_rating.drop(columns = ["DFG_per","FMwt","Stops1","Stops2","Team_possessions"])
rating1 = rating1
ratings2 = fs_final

player_stats = pd.concat((rating1,ratings2),axis = 1)
player_stats.fillna(0, inplace = True)

In [23]:
rand_input = input("Do you want to print the stats")


Do you want to print the stats5


In [53]:
with pd.ExcelWriter(file,engine ='openpyxl',mode = 'a') as writer:
    #data_frame.to_excel(writer,sheet_name = "BoxScores")
    player_stats.to_excel(writer,sheet_name='PlayerStats')
    #pd.DataFrame(m,columns = ["Match_list"]).to_excel(writer,sheet_name='Match_list')


